In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib as plb
import seaborn as sns
from scipy import stats
import statistics
import dabest as db

In [2]:
p1 = pd.read_csv('C:/Users/Emily/Documents/S1F1/S1fuPlate1.csv')
p1s = p1.iloc[range(6)]
p1s = p1s.melt(id_vars=['Let'], var_name= 'Col', value_name= 'Compound')
p1s['CPID'] = 'P1'

In [3]:
p2 = pd.read_csv('C:/Users/Emily/Documents/S1F1/S1fuPlate2.csv')
p2s = p2.iloc[range(6)]
p2s = p2s.melt(id_vars=['Let'], var_name= 'Col', value_name= 'Compound')
p2s['CPID'] = 'P2'

In [88]:
sub = ['Compound', 'Location']
ckey = pd.read_csv('C:/Users/Emily/Documents/S1F1/ckey.csv', index_col=0).drop(columns=['Let','Col'])
ckey_x = pd.read_csv('C:/Users/Emily/Documents/S1F1/D5P1.csv', usecols=sub)
ckey_x.rename(columns={'Location': 'Compound Well'}, inplace=True)
ckey_x['CPID'] = 'D5P1'
ckeys = ckey.append(ckey_x)

In [152]:
s1f1_md = pd.read_csv('C:/Users/Emily/Documents/S1F1/S1F1_metadata.csv', delimiter=',', 
                      encoding='utf-8-sig')

s1f1_md['Image ID'] = s1f1_md['Image ID'].str[:5] + '0' + s1f1_md['Image ID'].str[5:8]

s1f2_md = pd.read_csv('C:/Users/Emily/Documents/S1F2/S1F2_md.csv', delimiter=',', 
                 encoding='utf-8-sig')

all_md = s1f1_md.append(s1f2_md)
all_md.rename(columns={'-': 'Date'}, inplace=True)

In [153]:
all_md.head()

,Date,Recorder,Plate ID,Different strains in each well of a single assay plate? (Y/N),Different compounds in each well of a single assay plate? (Y/N),Compound library ID,Compound Well A,Compound Well B,Compound Well C,Compound Well D,...,Chemotaxis Start (24 hrs format),Chemotaxis End (24 hrs format),Image ID,Scanner Slot:,Scanner Slot,Scanner #,Strain Well A,Strain Well B,Strain Well C,Strain Well D
0,10/25/2021,Ehsan,S1F1_R1_01,NaN,NaN,D1P2,B2,B3,B4,B5,...,NaN,NaN,S1F1_001,1,NaN,3,CX10,CX10,CX10,CX10
1,10/25/2021,Ehsan,S1F1_R1_02,NaN,NaN,D1P2,C2,C3,C4,C5,...,NaN,NaN,S1F1_001,2,NaN,3,CX10,CX10,CX10,CX10
2,10/25/2021,Ehsan,S1F1_R1_03,NaN,NaN,D1P2,D2,D3,D4,D5,...,NaN,NaN,S1F1_001,3,NaN,3,CX10,CX10,CX10,CX10
3,10/25/2021,Ehsan,S1F1_R1_04,NaN,NaN,D1P2,E2,E3,E4,E5,...,NaN,NaN,S1F1_001,4,NaN,3,CX10,CX10,CX10,CX10
4,10/25/2021,Ehsan,S1F1_R1_05,NaN,NaN,D1P2,F2,F3,F4,F5,...,NaN,NaN,S1F1_002,1,NaN,3,CX10,CX10,CX10,CX10


In [154]:
def add_cipd(row):
    if row['Compound library ID'] != 'D5P1':
        pid = row['Compound library ID'][2:4]
    else:
        pid = 'D5P1'
    return pid

all_md['CPID'] = all_md.apply(lambda row: add_cipd(row), axis=1)


In [155]:
all_md['Scanner Slot:'] = all_md['Scanner Slot:'].apply(str)

In [156]:
df_obj = all_md.select_dtypes(['object', 'string'])
all_md = all_md[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
all_md = all_md.fillna('Empty')
all_md.tail()

,Date,Recorder,Plate ID,Compound library ID,Compound Well A,Compound Well B,Compound Well C,Compound Well D,Chemotaxis Start (24 hrs format),Chemotaxis End (24 hrs format),Image ID,Scanner Slot:,Strain Well A,Strain Well B,Strain Well C,Strain Well D,CPID
34,3/17/2022,Ehsan,S1F2_R3_11,D3P1,F2,F3,F4,F5,2:45,3:45,S1F2_009,3,GN960,GN960,GN960,GN960,P1
35,3/17/2022,Ehsan,S1F2_R3_12,D3P1,G2,G3,G4,G5,2:45,3:45,S1F2_009,4,GN960,GN960,GN960,GN960,P1
36,3/21/2022,EMILY,S1F2_R4_1,D4P1,F5,G3,G2,F4,10:55,11:55,S1F2_010,1,GN960,GN960,GN960,GN960,P1
37,3/21/2022,EMILY,S1F2_R4_2,D4P1,E2,C4,E4,B3,10:55,11:55,S1F2_010,2,GN960,GN960,GN960,GN960,P1
38,3/21/2022,EMILY,S1F2_R4_3,D4P2,D4,F2,Empty,Empty,10:55,11:55,S1F2_010,3,GN960,GN960,GN960,GN960,P2


In [157]:
def add_compoundA(row, compound_map):
    if row['Compound Well A'] != 'Empty':
        compound = compound_map.loc[
            (compound_map['CPID']==row['CPID']) & 
            (compound_map['Compound Well']==row['Compound Well A'])]['Compound']
        if len(compound)<0:
            return "Empty"
        else:
            return compound.values[0]
    else:
        return "Empty"
    
def add_compoundB(row, compound_map ):
    if row['Compound Well B'] != 'Empty':
        compound = compound_map.loc[
            (compound_map['CPID']==row['CPID']) & 
            (compound_map['Compound Well']==row['Compound Well B'])]['Compound']
        if len(compound)<0:
            return "Empty"
        else:
            return compound.values[0]
    else:
        return "Empty"

def add_compoundC(row, compound_map ):
    if row['Compound Well C'] != 'Empty':
        compound = compound_map.loc[
            (compound_map['CPID']==row['CPID']) & 
            (compound_map['Compound Well']==row['Compound Well C'])]['Compound']
        if len(compound)<0:
            return "Empty"
        else:
            return compound.values[0]
    else:
        return "Empty"

def add_compoundD(row, compound_map ):
    if row['Compound Well D'] != 'Empty':
        compound = compound_map.loc[
            (compound_map['CPID']==row['CPID']) & 
            (compound_map['Compound Well']==row['Compound Well D'])]['Compound']
        if len(compound)<0:
            return "Empty"
        else:
            return compound.values[0]
    else:
        return "Empty"


In [158]:
all_md['Compound A'] = all_md.apply(
    lambda row: add_compoundA(row, ckeys), axis=1)

all_md['Compound B'] = all_md.apply(
    lambda row: add_compoundB(row, ckeys), axis=1)
    
all_md['Compound C'] = all_md.apply(
    lambda row: add_compoundC(row, ckeys), axis=1)
    
all_md['Compound D'] = all_md.apply(
    lambda row: add_compoundD(row, ckeys), axis=1)

In [159]:
F1_ia_results = pd.read_csv('C:/Users/Emily/Documents/S1f1/124_ia/S1F1.csv', index_col=0).drop(columns=['Large Object'])
F2_ia_results = pd.read_csv('C:/Users/Emily/Documents/S1F2/S1F2_ia.csv', index_col=0).drop(columns=['Large Object'])
make_ups = pd.read_csv('C:/Users/Emily/Documents/S1F2/S1F2_ia.csv', index_col=0).drop(columns=['Large Object'])
all_ia = F1_ia_results.append(F2_ia_results)


In [160]:
broll = ['S1F1_b010', 'S1F1_b011', 'S1F1_b012']
all_ia= all_ia[~all_ia['File Name'].isin(broll)]
ia_results = all_ia.loc[all_ia['Total Worms'] >= 150]
ia_results.head()

,WellNo,Total Worms,Chemotaxis,Compound,Strain,File Name,Well width,Plate ID
1,1B,199.0,0.233533,NaN,NaN,S1F1_001,3044.0,NaN
2,1C,306.0,0.225352,NaN,NaN,S1F1_001,3044.0,NaN
3,1D,208.0,0.390374,NaN,NaN,S1F1_001,3057.0,NaN
4,2A,282.0,-0.019920,NaN,NaN,S1F1_001,3057.0,NaN
5,2B,239.0,0.171429,NaN,NaN,S1F1_001,3043.0,NaN


In [161]:
def add_PlateID(row, metadata):
    slotID = row['WellNo'][0]
    pid = metadata.loc[
        (metadata['Image ID']==row['File Name']) & 
        (metadata['Scanner Slot:']==slotID)]['Plate ID']

    if len(pid) == 0:
        print(row['File Name'])
        pass
    #print(pid)
    else:
        return pid.values[0]

ia_results['Plate ID'] = ia_results.apply(
    lambda row: add_PlateID(row, all_md), axis=1)

ia_results.head()

C:\ProgramData\Anaconda3\envs\neuroplant\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,WellNo,Total Worms,Chemotaxis,Compound,Strain,File Name,Well width,Plate ID
1,1B,199.0,0.233533,NaN,NaN,S1F1_001,3044.0,S1F1_R1_01
2,1C,306.0,0.225352,NaN,NaN,S1F1_001,3044.0,S1F1_R1_01
3,1D,208.0,0.390374,NaN,NaN,S1F1_001,3057.0,S1F1_R1_01
4,2A,282.0,-0.019920,NaN,NaN,S1F1_001,3057.0,S1F1_R1_02
5,2B,239.0,0.171429,NaN,NaN,S1F1_001,3043.0,S1F1_R1_02


In [162]:
#ia = pd.read_csv('D:/_2021_08_screen/S1fu/ub_md.csv', index_col=0)
#ia.head()

In [163]:
def add_Compound(row, metadata):
    wellID = row['WellNo'][1]

    if wellID == 'A':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound A']
    elif wellID == 'B':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound B']
    elif wellID == 'C':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound C']
    elif wellID == 'D':
        compound = metadata.loc[metadata['Plate ID']==row['Plate ID']]['Compound D']
    #print(compound)
    return compound.values[0]

ia_results['Compound'] = ia_results.apply(
    lambda row: add_Compound(row, all_md), axis=1)



C:\ProgramData\Anaconda3\envs\neuroplant\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [164]:
def add_Strain(row, metadata):
    slotID = row['WellNo'][0]
    strain = metadata.loc[
        (metadata['Plate ID']==row['Plate ID']) & 
        (metadata['Scanner Slot:']==slotID)]['Strain Well A ']
    
    if len(strain) == 0:
        return 'Empty'
    else:
        return strain.values[0]

ia_results['Strain'] = ia_results.apply(
    lambda row: add_Strain(row, all_md), axis=1)


C:\ProgramData\Anaconda3\envs\neuroplant\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [165]:
filtered = ia_results.loc[ia_results['Total Worms'] >= 150]
print(len(filtered))

487


In [166]:
filtered = filtered.loc[filtered['Compound'] != 'Empty']

In [135]:

g = filtered.groupby(['Compound', 'Strain']).count()
g.to_csv('C:/Users/Emily/Documents/follow_up_counts.csv')

In [139]:
S1F1_worm_locs_foldr = plb.Path('C:/Users/Emily/Documents/S1f1/124_ia/')
S1F2_worm_locs_foldr = plb.Path('C:/Users/Emily/Documents/S1F2/')

In [140]:
# 1 inch = 25.4mm
mm = 25.4
# 1200 pixels per 25.4mm
px_mm = 1200/mm

#The following transforms the worm location data so that it is respective to the start zone
# The start zone is at the center of the image; Start Zone = 0mm
#Worms with positive values are closer to the compound, negative values are away from the compound


In [141]:
def get_worm_locs(row, wrms, result_dict, strain): 

    fname = row['File Name']
    wellnum = row['WellNo']

    if row['Strain'] == strain:
        loc_fname =  wrms.joinpath('loc_' + fname + '_' + wellnum + '.csv')
        temp = pd.read_csv(loc_fname)
        compound = row['Compound']
        xs = temp['X']
        #xs = list(temp['centroid-1'])
        if compound in result_dict:
            result_dict[compound] = result_dict[compound].append(xs)
            result_dict[compound].reset_index(inplace=True, drop=True)
            #result_dict[compound] = result_dict[compound]+xs
        else:
            result_dict[compound]=xs

    return result_dict


In [144]:
# Need to create an empty dictionary to hold the values
results_dict = {}
for index, row in filtered.iterrows():
    pooled = get_worm_locs(row, S1F2_worm_locs_foldr, results_dict, 'GN960')
GN_pooled_df = pd.DataFrame.from_dict(pooled)
GN960_df = GN_pooled_df.apply(lambda x: -(x/px_mm)+32.5)

GN960_df = GN960_df.drop(columns = ['Rosmarinic acid'])

In [146]:
# Need to create an empty dictionary to hold the values
results_dict = {}
for index, row in filtered.iterrows():
    CX_pooled = get_worm_locs(row, S1F1_worm_locs_foldr, results_dict, 'CX10')
CX_pooled_df = pd.DataFrame.from_dict(CX_pooled)
CXmm_df = CX_pooled_df.apply(lambda x: -(x/px_mm)+32.5)


In [148]:
# Need to create an empty dictionary to hold the values
results_dict = {}
for index, row in filtered.iterrows():
    PR_pooled = get_worm_locs(row, S1F1_worm_locs_foldr, results_dict, 'PR678')
PR_pooled_df = pd.DataFrame.from_dict(PR_pooled)
PRmm_df = PR_pooled_df.apply(lambda x: -(x/px_mm)+32.5)

In [3]:
#PRmm_df.to_csv('D:/_2021_08_screen/S1fu/150_PR678.csv')
#CXmm_df.to_csv('D:/_2021_08_screen/S1fu/150_CX10.csv')

#PRmm_df = pd.read_csv('D:/_2021_08_screen/S1fu/allReps_over150/150_PR678.csv', index_col=0)
#CXmm_df = pd.read_csv('D:/_2021_08_screen/S1fu/allReps_over150/150_CX10.csv', index_col=0)
 

In [23]:
GN960_df_mm = mm_obj.mean_diff.results
GN960_df_mm.to_csv('C:/Users/Emily/Desktop/current_analysis/S1F2/GN960_H2O_mdiff.csv')

In [24]:
dmso = pd.read_csv('C:/Users/Emily/Desktop/current_analysis/S1F2/GN960_mdiff.csv')
all_refs = dmso.append(GN960_df_mm)
all_refs.to_csv('C:/Users/Emily/Desktop/current_analysis/S1F2/GN960_mdiff_h2odmso.csv')

In [88]:
mm_df.to_csv('C:/Users/Emily/Desktop/current_analysis/S1F2/GN960_xs.csv')

In [4]:
cols = list(CXmm_df.columns)
cols.insert(0, cols.pop(cols.index('DMSO')))
print(cols)

['DMSO', 'Carnosol', 'Limonin', 'H2O', 'Isoquinoline', '2-Nonylquinolin-4(1H)-one', 'Lapachol', 'Acetophenone', '1-octanol', 'Furfural', 'Camphor', 'Thiophene', 'Solasodine', 'Salvinorin A Propionate', 'Sinomenine hydrochloride', 'Diacetyl', '2,5-Dihydroxybenzoic acid', 'Coumaran', 'p-Tolualdehyde', 'Isoamyl alcohol', '2,3-Dihydrobenzofuran', 'Piperitenone', 'Guaiazulene', 'Oleanolic Acid', 'Sabinene', 'Spinosad', 'L-Mimosine', 'Phenylacetylene', 'Rosmarinic acid', '2-nonanone', 'Daucosterol', 'Leonurine', 'Cinnamyl Alcohol', '(-)-Huperzine A', '2-Methyl-1-butanol', 'Paeoniflorin', 'Ursolic acid', 'Ethyl palmitate', 'Piperonyl Alcohol', 'Anisole', 'α-Phellandrene', 'Ethyl p-methoxycinnamate', 'Phytol', 'Methyl palmitate', 'Ellagic acid']


In [49]:
cx10_mm_obj = db.load(CXmm_df, idx=(cols))
cx10_df_mm =cx10_mm_obj.mean_diff.results
cx10_df_mm.to_csv('D:/_2021_08_screen/S1fu/150_CX10_meandif.csv')

In [50]:
pr678_mm_obj = db.load(PRmm_df, idx=(cols))
pr678_df_mm = pr678_mm_obj.mean_diff.results
pr678_df_mm.to_csv('D:/_2021_08_screen/S1fu/150_PR678_meandif.csv')

In [5]:
def mean_diff_calc(df, ref):
    cols = list(df.columns)
    cols.insert(0, cols.pop(cols.index(ref)))
    db_obj = db.load(df, idx=(cols))
    results_df = db_obj.mean_diff.results
    return results_df

In [78]:

pr_ref_df = pr678_df_mm[pr678_df_mm['test'].isin(refs)]

pr_ref_df['test'] = pd.Categorical(pr_ref_df['test'], refs)
pr_ref_df

C:\ProgramData\Anaconda3\envs\neuroplant\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,control,test,control_N,test_N,effect_size,is_paired,difference,ci,bca_low,bca_high,...,resamples,random_seed,pvalue_permutation,permutation_count,pvalue_welch,statistic_welch,pvalue_students_t,statistic_students_t,pvalue_mann_whitney,statistic_mann_whitney
1,DMSO,1-octanol,851,668,mean difference,False,-3.953032,95,-5.293354,-2.707069,...,5000,12345,0.0,5000,1.940105e-09,6.040730,2.570381e-09,5.992922,4.766293e-09,333919.0
5,DMSO,2-nonanone,851,767,mean difference,False,-6.325735,95,-7.514221,-5.133704,...,5000,12345,0.0,5000,1.467615e-24,10.398968,3.142214e-24,10.321014,9.336639e-23,418501.0
14,DMSO,Diacetyl,851,710,mean difference,False,9.447680,95,8.162622,10.738549,...,5000,12345,0.0,5000,1.732651e-44,-14.455560,2.883483e-44,-14.406118,2.001945e-41,182543.0
20,DMSO,H2O,851,523,mean difference,False,3.815697,95,2.391063,5.156433,...,5000,12345,0.0,5000,7.866413e-08,-5.405207,1.104554e-07,-5.336983,1.030748e-06,187646.0
22,DMSO,Isoamyl alcohol,851,889,mean difference,False,3.360917,95,2.089409,4.599195,...,5000,12345,0.0,5000,1.156460e-07,-5.322538,1.161310e-07,-5.321735,5.691992e-07,325871.0


In [79]:
cx_ref_df = cx10_df_mm[cx10_df_mm['test'].isin(refs)]

cx_ref_df['test'] = pd.Categorical(cx_ref_df['test'], refs)
cx_ref_df

C:\ProgramData\Anaconda3\envs\neuroplant\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,control,test,control_N,test_N,effect_size,is_paired,difference,ci,bca_low,bca_high,...,resamples,random_seed,pvalue_permutation,permutation_count,pvalue_welch,statistic_welch,pvalue_students_t,statistic_students_t,pvalue_mann_whitney,statistic_mann_whitney
1,DMSO,1-octanol,936,806,mean difference,False,-1.173355,95,-2.655425,0.347436,...,5000,12345,0.1274,5000,1.266346e-01,1.528241,1.297979e-01,1.515617,2.396278e-01,389518.0
5,DMSO,2-nonanone,936,861,mean difference,False,-3.744640,95,-5.228813,-2.159922,...,5000,12345,0.0000,5000,1.834243e-06,4.786742,1.908140e-06,4.778675,2.985983e-06,454285.0
14,DMSO,Diacetyl,936,861,mean difference,False,3.997838,95,2.467943,5.489077,...,5000,12345,0.0000,5000,4.239087e-07,-5.076741,4.366735e-07,-5.070958,5.360118e-07,347862.0
20,DMSO,H2O,936,909,mean difference,False,2.071254,95,0.553972,3.540891,...,5000,12345,0.0078,5000,6.312916e-03,-2.734214,6.373819e-03,-2.731031,5.258723e-03,393484.0
22,DMSO,Isoamyl alcohol,936,711,mean difference,False,11.387340,95,9.915606,12.882325,...,5000,12345,0.0000,5000,5.840411e-49,-15.207184,2.482915e-46,-14.748838,1.347746e-41,203582.0


In [80]:
combined = pr_ref_df.append(cx_ref_df)
combined

,control,test,control_N,test_N,effect_size,is_paired,difference,ci,bca_low,bca_high,...,resamples,random_seed,pvalue_permutation,permutation_count,pvalue_welch,statistic_welch,pvalue_students_t,statistic_students_t,pvalue_mann_whitney,statistic_mann_whitney
1,DMSO,1-octanol,851,668,mean difference,False,-3.953032,95,-5.293354,-2.707069,...,5000,12345,0.0000,5000,1.940105e-09,6.040730,2.570381e-09,5.992922,4.766293e-09,333919.0
5,DMSO,2-nonanone,851,767,mean difference,False,-6.325735,95,-7.514221,-5.133704,...,5000,12345,0.0000,5000,1.467615e-24,10.398968,3.142214e-24,10.321014,9.336639e-23,418501.0
14,DMSO,Diacetyl,851,710,mean difference,False,9.447680,95,8.162622,10.738549,...,5000,12345,0.0000,5000,1.732651e-44,-14.455560,2.883483e-44,-14.406118,2.001945e-41,182543.0
20,DMSO,H2O,851,523,mean difference,False,3.815697,95,2.391063,5.156433,...,5000,12345,0.0000,5000,7.866413e-08,-5.405207,1.104554e-07,-5.336983,1.030748e-06,187646.0
22,DMSO,Isoamyl alcohol,851,889,mean difference,False,3.360917,95,2.089409,4.599195,...,5000,12345,0.0000,5000,1.156460e-07,-5.322538,1.161310e-07,-5.321735,5.691992e-07,325871.0
1,DMSO,1-octanol,936,806,mean difference,False,-1.173355,95,-2.655425,0.347436,...,5000,12345,0.1274,5000,1.266346e-01,1.528241,1.297979e-01,1.515617,2.396278e-01,389518.0
5,DMSO,2-nonanone,936,861,mean difference,False,-3.744640,95,-5.228813,-2.159922,...,5000,12345,0.0000,5000,1.834243e-06,4.786742,1.908140e-06,4.778675,2.985983e-06,454285.0
14,DMSO,Diacetyl,936,861,mean difference,False,3.997838,95,2.467943,5.489077,...,5000,12345,0.0000,5000,4.239087e-07,-5.076741,4.366735e-07,-5.070958,5.360118e-07,347862.0
20,DMSO,H2O,936,909,mean difference,False,2.071254,95,0.553972,3.540891,...,5000,12345,0.0078,5000,6.312916e-03,-2.734214,6.373819e-03,-2.731031,5.258723e-03,393484.0
22,DMSO,Isoamyl alcohol,936,711,mean difference,False,11.387340,95,9.915606,12.882325,...,5000,12345,0.0000,5000,5.840411e-49,-15.207184,2.482915e-46,-14.748838,1.347746e-41,203582.0
